# MLflow Model Registry — Managing Versions and Aliases

This notebook shows how to register models, assign aliases, and evaluate variants using MLflow’s Model Registry.

Sections
- Configure Tracking URI
- Interacting with the MLflow Tracking and Registry APIs
- Create Client and List Experiments
- Query Top Runs by Metric
- Interacting with the Model Registry
- Register Model Versions
- List Registered Models
- Working with Aliases
- Load Model by Alias
- Troubleshooting: Missing Model Artifacts
- Cleanup and Fix Aliases
- Helper Functions
- Load Test Data
- Download and Load Preprocessor
- Vectorize Test Features
- Define Target Variable
- Evaluate Champion Model
- Evaluate Challenger Model
- Promote the new Champion

### Configure Tracking URI

- Set the tracking backend location (SQLite file in this example)
- Use the same URI for both tracking and registry operations


In [15]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

- In this example, the URI points to a local SQLite database file `mlflow.db`.
- The format `sqlite:///mlflow.db` means:
  - `sqlite`: SQLite database engine
  - `///mlflow.db`: path to the database file in the current directory

## Interacting with the MLflow Tracking and Registry APIs

The `MlflowClient` lets us interact with:
- The MLflow Tracking Server (experiments and runs)
- The MLflow Model Registry (registered models and versions)

Instantiate it by providing a tracking URI (and optionally a separate registry URI).

### Create Client and List Experiments

- Instantiate `MlflowClient`
- Verify connection by listing experiments


In [16]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

# In MLflow 2.x, use search_experiments() instead of list_experiments()
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2', creation_time=1761832181456, experiment_id='2', last_update_time=1761832181456, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1761828359873, experiment_id='1', last_update_time=1761828359873, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/0', creation_time=1761828359867, experiment_id='0', last_update_time=1761828359867, lifecycle_stage='active', name='Default', tags={}>]

In [17]:
# # Create a new experiment named "my-cool-experiment" in the MLflow tracking server.
# # If an experiment with this name already exists, this will raise an exception.
# client.create_experiment(name="my-cool-experiment")

Let's check the latest versions for the experiment with id `1`...

### Query Top Runs by Metric

- Filter by `rmse < 7` and sort ascending
- Fetch a limited number of best runs


In [76]:
# Search for the top 5 runs in experiment '1' where the RMSE metric is less than 7,
# sorted in ascending order of RMSE. Only ACTIVE runs are considered.
# 'runs' will be a list of Run objects matching this criteria.

from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [77]:
runs

[<Run: data=<RunData: metrics={'rmse': 4.569990569874923}, params={'learning_rate': '0.34675437333267856',
  'max_depth': '44',
  'min_child_weight': '0.5453440445225199',
  'objective': 'reg:squarederror',
  'reg_alpha': '0.01288008035321842',
  'reg_lambda': '0.13210782702114726',
  'seed': '42'}, tags={'mlflow.runName': 'sassy-loon-195',
  'mlflow.source.name': '/home/codespace/anaconda3/envs/experiment-tracking/lib/python3.9/site-packages/ipykernel_launcher.py',
  'mlflow.source.type': 'LOCAL',
  'mlflow.user': 'codespace',
  'model': 'xgboost'}>, info=<RunInfo: artifact_uri='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/0ea9354f03c444e394092b22535a37ec/artifacts', end_time=1761833418236, experiment_id='1', lifecycle_stage='active', run_id='0ea9354f03c444e394092b22535a37ec', run_name='sassy-loon-195', run_uuid='0ea9354f03c444e394092b22535a37ec', start_time=1761833224531, status='FINISHED', user_id='codespace'>, inputs=<RunInputs: dataset_inputs=[]>>,
 <Run: data=<RunDa

In [78]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 0ea9354f03c444e394092b22535a37ec, rmse: 4.5700
run id: cd20c2a4aa2f494a9fe17b9a902e761d, rmse: 4.5816
run id: ee5b836f564b4edaaa2a3b5711977c4b, rmse: 4.6090
run id: 4cbe44e7872b49238546404fd3f0b5f4, rmse: 4.6352
run id: a349d7ef8aaa4359adbf8f995571efc5, rmse: 4.6494


## Interacting with the Model Registry

In this section we will use the `MlflowClient` to:

1. Register new versions of the model `nyc-taxi-regressor`
2. Retrieve the latest versions and inspect their metadata
3. Manage aliases (e.g., `champion`, `challenger`) instead of stages

### Ensure Tracking URI for Registry Operations

- Set tracking URI before interacting with the registry
- Keeps tracking and registry calls consistent


In [21]:
# Registering and managing model versions in the MLflow Model Registry

# The following code block ensures we are using the correct MLflow tracking server URI.
# This is necessary to connect to the MLflow backend for logging and model registry operations.
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

### Register Model Versions

- Register artifacts from specific runs to create new versions
- Repeat to create multiple versions for comparison


In [ ]:
run_id = "0ea9354f03c444e394092b22535a37ec"

Successfully registered model 'nyc-taxi-regressor'.
Created version '1' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1761834931853, current_stage='None', description=None, last_updated_timestamp=1761834931853, name='nyc-taxi-regressor', run_id='0ea9354f03c444e394092b22535a37ec', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/0ea9354f03c444e394092b22535a37ec/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [ ]:
# model_uri = f"runs:/{run_id}/model"
# mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

In [ ]:
# # To create version 2, you need to register a new model artifact with the same registered model name.
# # The following code creates a new version by registering the model artifact from a different run.
# run_id2 = "cd20c2a4aa2f494a9fe17b9a902e761d"
# model_uri = f"runs:/{run_id2}/model"
# mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")


Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '2' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1761835815723, current_stage='None', description=None, last_updated_timestamp=1761835815723, name='nyc-taxi-regressor', run_id='cd20c2a4aa2f494a9fe17b9a902e761d', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/cd20c2a4aa2f494a9fe17b9a902e761d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

### List Registered Models

- Inspect available registered models and versions


In [31]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1761834931839, description=None, last_updated_timestamp=1761835815723, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1761834931853, current_stage='Production', description=None, last_updated_timestamp=1761835348391, name='nyc-taxi-regressor', run_id='0ea9354f03c444e394092b22535a37ec', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/0ea9354f03c444e394092b22535a37ec/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>,
  <ModelVersion: aliases=[], creation_timestamp=1761835815723, current_stage='None', description=None, last_updated_timestamp=1761835815723, name='nyc-taxi-regressor', run_id='cd20c2a4aa2f494a9fe17b9a902e761d', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/cd20c2a4aa2f494a9fe17b9a902e761d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>], name='nyc-taxi-regressor'

In [64]:
# Retrieve the name of the registered model we want to interact with
model_name = "nyc-taxi-regressor"

# Get the latest version(s) of the specified model using the MLflow client.
# This allows us to inspect the available versions and their current stages (e.g., None, Staging, Production, etc.).
latest_versions = client.get_latest_versions(name=model_name)

/tmp/ipykernel_167100/82709836.py:6: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


### Working with Aliases

- Assign aliases like `champion` and `challenger`
- Retrieve versions via aliases for inference and management


In [70]:
# Set an alias
client.set_registered_model_alias(
    name="nyc-taxi-regressor",
    alias="champion",  # or "production", "staging", "challenger", etc.
    version="1"
)

In [67]:
# Get version by alias
champion_version = client.get_model_version_by_alias(
    name="nyc-taxi-regressor",
    alias="champion"
)

In [68]:
champion_version

<ModelVersion: aliases=['champion'], creation_timestamp=1761834931853, current_stage='Production', description=None, last_updated_timestamp=1761836799622, name='nyc-taxi-regressor', run_id='0ea9354f03c444e394092b22535a37ec', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/0ea9354f03c444e394092b22535a37ec/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [74]:
import mlflow
from mlflow.tracking import MlflowClient

print("Tracking URI:", mlflow.get_tracking_uri())

client = MlflowClient()
mv = client.get_model_version_by_alias("nyc-taxi-regressor", "champion")
print("Alias -> version:", mv.version)
print("Alias -> source:", mv.source)  # where MLflow expects the model files
print("Alias -> run_id:", mv.run_id)


Tracking URI: sqlite:///mlflow.db
Alias -> version: 1
Alias -> source: /workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/0ea9354f03c444e394092b22535a37ec/artifacts/model
Alias -> run_id: 0ea9354f03c444e394092b22535a37ec


### Load Model by Alias

- Use `models:/<name>@<alias>` to resolve to a specific version


In [45]:
mlflow.pyfunc.load_model(f"models:/nyc-taxi-regressor@champion")

OSError: No such file or directory: '/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/0ea9354f03c444e394092b22535a37ec/artifacts/model/.'

### Troubleshooting: Missing Model Artifacts

- Ensure the registered model’s run has the actual model files
- Re-register from the correct run or adjust aliases accordingly


* The error occurred because **model artifacts are missing**.
* The registered model points to **run ID `0ea9354f03c444e394092b22535a37ec`**, but that run **doesn’t exist** or has **no saved model files**.
* The real model artifacts are in a **different run**

### Cleanup and Fix Aliases

- Remove incorrect registrations
- Set aliases (`champion`, `challenger`) to correct versions
- Optionally update descriptions


In [75]:
# Delete the incorrectly registered model first
client.delete_registered_model(name="nyc-taxi-regressor")

In [82]:
client.set_registered_model_alias(
    name="nyc-taxi-test",
    alias="champion",  # or "production", "staging", "challenger", etc.
    version="1"
)

In [85]:
model_prod = mlflow.pyfunc.load_model("models:/nyc-taxi-test@champion")

In [83]:
# Update the description separately
client.update_model_version(
    name="nyc-taxi-test",
    version="1",
    description="This is the champion version of the model."
)

<ModelVersion: aliases=['champion', 'test'], creation_timestamp=1761831879302, current_stage='None', description='This is the champion version of the model.', last_updated_timestamp=1761837885864, name='nyc-taxi-test', run_id='712e9c4fb3294a75bd60b15f76102062', run_link='', source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/712e9c4fb3294a75bd60b15f76102062/artifacts/models_mlflow', status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=1>

## Comparing versions and selecting the new “Champion” model

In this section, we’ll retrieve models registered in the MLflow Model Registry and compare their performance on an unseen test set.

This simulates a real-world scenario where a deployment engineer evaluates whether the current **production model** (the **“champion”**) should be replaced by a newer **“challenger”** model.

1. **Load the test dataset**: Use the [NYC Yellow Taxi data from March 2021](https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-03.parquet) for evaluation.
2. **Load the preprocessing artifact**: Download the saved `DictVectorizer` (logged as an artifact in MLflow) and load it using `pickle`.
3. **Preprocess the test data**: Transform the raw test dataset with the loaded `DictVectorizer` so it matches the training feature format.
4. **Compare model performance**

	* Load the **current production model** using its alias:

	```python
	model_prod = mlflow.pyfunc.load_model("models:/nyc-taxi-test@champion")
	```

	* Load the **candidate model** using another alias or version, for example:

	```python
	model_stage = mlflow.pyfunc.load_model("models:/nyc-taxi-test@challenger")
	```

	* Evaluate both models on the test set and compare their RMSE (or another metric).

5. **Promote the better model**
	 If the challenger outperforms the champion, update the alias assignments.

**NOTE**:

MLflow’s Model Registry **does not automatically deploy models** when you change an alias.

Setting an alias (like `"champion"` or `"challenger"`) simply labels which version holds that role.

Actual deployment to production environments should be handled by your **CI/CD pipeline or orchestration tool** (e.g., GitHub Actions, Airflow, or Azure ML pipelines).


### Helper Functions

- `read_dataframe`: load and clean dataset
- `preprocess`: build features with `PU_DO` + `trip_distance`
- `test_model`: load by alias and compute RMSE


In [98]:
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

def read_dataframe(filename):
    # Load the Parquet file (used for faster I/O and smaller size)
    df = pd.read_parquet(filename)

    # Convert pickup and dropoff columns from string/object to datetime
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

    # Calculate trip duration in minutes
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    # Keep only reasonable trips (between 1 and 60 minutes)
    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    # Convert pickup/dropoff location IDs to string for categorical processing
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    # Create a combined pickup–dropoff feature (e.g., "138_265")
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']

    # Define which features are categorical and which are numerical
    categorical = ['PU_DO']
    numerical = ['trip_distance']

    # Convert the feature columns into a list of dictionaries
    # (each row → a dict of feature_name → value)
    data_dicts = df[categorical + numerical].to_dict(orient='records')

    # Transform the dictionaries into the same vectorized format used during training
    return dv.transform(data_dicts)


def test_model(name, alias, X_test, y_test):
    # Load the model from the MLflow Model Registry using an alias (e.g., "champion", "challenger")
    model = mlflow.pyfunc.load_model(f"models:/{name}@{alias}")

    # Generate predictions on the test feature matrix
    y_pred = model.predict(X_test)

    # Compute the Root Mean Squared Error (RMSE) - a lower value means better performance
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    return {"rmse": rmse}

### Load Test Data

- Use NYC Yellow Taxi March 2021 as test set
- Apply the same preprocessing as training


In [88]:
df = read_dataframe("/workspaces/mlops-zoomcamp/data/yellow_tripdata_2021-03.parquet")

In [ ]:
champion_version = client.get_model_version_by_alias(
    name="nyc-taxi-test",
    alias="champion"
)

### Download and Load Preprocessor

- Download the `preprocessor` artifact from the run
- Load the `DictVectorizer` from the artifact directory


In [89]:
run_id = "712e9c4fb3294a75bd60b15f76102062"
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/workspaces/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [90]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

### Vectorize Test Features

- Transform raw test dataframe using the saved `DictVectorizer`


In [91]:
X_test = preprocess(df, dv)

### Define Target Variable

- Predict the `duration` in minutes from the test data


In [92]:
target = "duration"
y_test = df[target].values

### Set up aliases for model comparison

Before testing, let's set up two aliases:
- **champion**: The current production model (version 1)
- **challenger**: A candidate model to evaluate (you can point this to version 2 or another version)


### Evaluate Champion Model

- Load the `champion` alias and compute RMSE on the test set


In [99]:
%time test_model(name="nyc-taxi-test", alias="champion", X_test=X_test, y_test=y_test)

CPU times: user 1min 22s, sys: 103 ms, total: 1min 22s
Wall time: 21.9 s


{'rmse': np.float64(4.472992694545769)}

### Evaluate Challenger Model

- Load the challenger model and evaluate on the same test set
- Compare RMSE against the champion


In [ ]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 6.94 s, sys: 216 ms, total: 7.16 s
Wall time: 7.28 s


{'rmse': 6.881555517147188}

### Promote the new Champion (update alias)

- Point `champion` alias to the best-performing version
- This updates the registry label; deployment is handled externally


In [ ]:
# Update the "champion" alias to point to the better performing model
# This replaces the old stage-based approach (transition_model_version_stage)
client.set_registered_model_alias(
    name="nyc-taxi-test",
    alias="champion",  # Promote this version to champion (production)
    version="1"  # Update this to the version number of the better model
)

<ModelVersion: creation_timestamp=1652971637398, current_stage='Production', description='The model version 4 was transitioned to Staging on 2022-05-19', last_updated_timestamp=1652972763255, name='nyc-taxi-regressor', run_id='b8904012c84343b5bf8ee72aa8f0f402', run_link=None, source='./mlruns/1/b8904012c84343b5bf8ee72aa8f0f402/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>